In [ ]:
import numpy as np

In [ ]:
with open('./datasets/emoji/train_embedding.csv', 'r') as input:
    embeds = input.read().split('\n')
    embeds = [ [ float(i) for i in embed.split(',') ] for embed in embeds if embed is not '']

In [ ]:
embeds = np.array(embeds)

In [ ]:
labels = np.loadtxt('./datasets/emoji/train_labels.txt')

In [ ]:
data_size = len(labels)

In [ ]:
embeds = embeds[0:len(labels), :]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(768, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 20)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
#         x = self.fc3(x)
        return x

In [ ]:
net = Net().to(device)
gpu_labels = torch.tensor(labels).to(device)
gpu_embeds = torch.tensor(embeds, dtype=torch.float32).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
batch_size = 32
for epoch in range(200):
    running_loss = 0.0
    for i in range( (data_size-1) // batch_size + 1 ):
        inputs = gpu_embeds[i*batch_size : min( (i+1)*batch_size, data_size), :]
        standards = gpu_labels[i*batch_size : min( (i+1)*batch_size, data_size)]
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
#         _, predicted = torch.max(outputs.data, 1)
        loss = criterion(outputs.float(), standards.long())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10000 == 9999:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10000))
            running_loss = 0.0

In [ ]:
print('Finished Training')

In [ ]:
NETPATH = './emoji_net.pth'
torch.save(net.state_dict(), NETPATH)

In [ ]:
correct = 0

with open('./datasets/emoji/test_embedding.csv', 'r') as input:
    embeds = input.read().split('\n')
    embeds = [ [ float(i) for i in embed.split(',') ] for embed in embeds if embed is not '']
    
embeds = np.array(embeds)
labels = np.loadtxt('./datasets/emoji/train_labels.txt')
data_size = len(labels)
embeds = embeds[0:len(labels), :]
    
gpu_labels_test = torch.tensor(labels).to(device)
gpu_embeds_test = torch.tensor(embeds, dtype=torch.float32).to(device)

with torch.no_grad():
    outputs = net(gpu_embeds_test)
    _, predicted = torch.max(outputs.data, 1)
    correct += (predicted == gpu_labels_test).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / data_size))